In [17]:
# you can skip it, I just like interactive charts
%matplotlib Qt

In [42]:
import numpy as np
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.wcs import WCS
from astropy.visualization.wcsaxes import SphericalCircle
from astropy.io import fits
import pandas as pd
# nice progressbar
from tqdm import tqdm

In [3]:
data = pd.read_csv("gaia.csv")
data = data.dropna(subset=['Gmag'])
mask = (data['Gmag'] > 0) & (data['Gmag'] < 200)
data = data[mask]
coords = SkyCoord(ra=data["RA_ICRS"], dec=data["DE_ICRS"], frame='icrs', unit=u.degree)

#### Grid

I create a rectangular grid. The boundaries are minimal and maximal RA and DEC from the given catalogue.

In [4]:
# grid_step = 1 # in arcmin
# ra_grid = np.arange(coords.ra.min().to(u.arcmin).value, coords.ra.max().to(u.arcmin).value, grid_step)
# dec_grid = np.arange(coords.dec.min().to(u.arcmin).value, coords.dec.max().to(u.arcmin).value, grid_step)
# ra_v, dec_v = np.meshgrid(ra_grid, dec_grid)
# # now grid is a 2-d array, each element is SkyCoord (ra,dec)
# grid = SkyCoord(ra_v, dec_v, unit=u.arcmin)

In [5]:
def create_wcs(center: SkyCoord, cdelt=1*u.arcmin, size=1*u.deg):
    pixsize = int((size.to(cdelt.unit)/cdelt).value)
    
    wcs = WCS(naxis=2)
    wcs.wcs.cdelt = [-cdelt.value, cdelt.value]
    wcs.wcs.cunit = [cdelt.unit, cdelt.unit]
    wcs.wcs.ctype = ["RA", "DEC"]
    wcs.wcs.crpix = [pixsize/2., pixsize/2.]
    wcs.wcs.crval = [center.ra.to(cdelt.unit).value, center.dec.to(cdelt.unit).value]
    wcs.pixel_shape = [pixsize, pixsize]
    return(wcs)

In [36]:
center = SkyCoord('03:29:11.28 +31:18:36.00', unit=(u.hourangle, u.deg))
# center = SkyCoord(ra_grid.mean(), dec_grid.mean(), unit=(u.arcsec, u.arcsec))
cdelt = 1 * u.arcmin
wcs = create_wcs(center, cdelt, size=2.5*u.deg)

#### Sourse density map

In [40]:
density = np.zeros(wcs.pixel_shape, dtype=float)

imax, jmax = wcs.pixel_shape

# it takes about 40s on my computer with density_window=5
density_window = 10*u.arcmin # diameter of region in which we calculate density
for i in tqdm(range(imax)):
    for j in range(jmax):
        mask = wcs.pixel_to_world(j, i).separation(coords) < density_window
        density[i,j] = len(mask[mask])

100%|█████████████████████████████████████████| 150/150 [01:08<00:00,  2.20it/s]


In [43]:
fits.PrimaryHDU(density, header=wcs.to_header()).writeto('GAIA_source_density.fits')

In [44]:
density = fits.getdata('GAIA_source_density.fits')
wcs = WCS(fits.getheader('GAIA_source_density.fits'))

In [45]:
# need to inverse array so that it cossesponds to the aladin axis orientation
fig1, ax1 = plt.subplots(subplot_kw=dict(projection=wcs))
# ax.imshow(density[:,::-1], origin='lower')
ax1.imshow(density, origin='lower')
ax1.scatter(x, y, s=1, marker=',', color='red')

In [37]:
x, y = wcs.world_to_pixel(coords)
map_dens, _, _ = np.histogram2d(y, x, wcs.pixel_shape)

In [39]:
# need to inverse array so that it cossesponds to the aladin axis orientation
fig1, ax1 = plt.subplots(subplot_kw=dict(projection=wcs))
# ax.imshow(density[:,::-1], origin='lower')
ax1.imshow(map_dens, origin='lower')
ax1.scatter(x, y, s=1, marker=',', color='red')

In [51]:
density_dis = density.flatten()
density_dis = density_dis[density_dis !=0]
n, num = np.histogram(density_dis, bins=10)
num = (num[:-1] + num[1:])/2
plt.figure()
plt.plot(num, n)

### Select region

In [10]:
def stars_from_region(data, coords, center, size):
    mask = center.separation(coords) < size
    return data[mask], coords[mask]

In [11]:
def hist_from_data(data, bins, min_mag=16, max_mag=20):
    N, m = np.histogram(data['Gmag'], bins=bins)
    m = (m[:-1] + m[1:])/2
    N = np.log10(np.cumsum(N))
    mask = (m > min_mag) & (m < max_mag)
    mask2 = (data['Gmag'] > min_mag) & (data['Gmag'] < max_mag)
    return N[mask], m[mask], len(mask2[mask2])

## Source histogram

In [12]:
fig1, ax1 = plt.subplots(subplot_kw=dict(projection=wcs))
fig2, ax2 = plt.subplots()
ax1.imshow(map_dens, origin='lower')
# r = SphericalCircle((center.ra, center.dec), 1*u.deg,
#                      edgecolor='green', facecolor='none',
#                      transform=ax1.get_transform('fk5'))
# ax1.add_patch(r)
reg_size = 16*u.arcmin

In [13]:
regs = ['03:28:07.84 +31:59:21.2',
        '03:26:24.14 +31:06:01.1',
        '03:30:36.56 +31:11:56.6',
        '03:31:27.32 +31:42:56.1']
b = []
n = []
for r in regs:
    reg_cen = SkyCoord(r, unit=(u.hourangle, u.deg))
    reg_data, reg_coords = stars_from_region(data, coords, reg_cen, reg_size)
    logN, m, tot = hist_from_data(reg_data, 20)
    print(tot)
    n.append(tot)
    p = np.polynomial.Polynomial.fit(m, logN, 1)
    print(p(1) - p(0))
    b.append(p(1) - p(0))
    ax2.plot(m, logN, 'k')
    ax2.plot(m, p(m), 'b')
    r = SphericalCircle((reg_cen.ra, reg_cen.dec), reg_size,
                     edgecolor='yellow', facecolor='none',
                     transform=ax1.get_transform('fk5'))
    ax1.add_patch(r)
b = np.average(b)
n = np.average(n)

1031
0.23261054744944776
867
0.22310624320557615
978
0.23122991088917155
923
0.25851261910228907


In [14]:
regs = ['03:29:00 +31:27:0',
        '03:29:00 +31:27:0']
for r in regs:
    reg_cen = SkyCoord(r, unit=(u.hourangle, u.deg))
    reg_data, reg_coords = stars_from_region(data, coords, reg_cen, reg_size)
    logN, m, tot = hist_from_data(reg_data, 20)
    p = np.polynomial.Polynomial.fit(m, logN, 1)
    print(p(1) - p(0))
    ax2.plot(m, logN, 'r')
    ax2.plot(m, p(m), 'b')
    r = SphericalCircle((reg_cen.ra, reg_cen.dec), reg_size,
                     edgecolor='red', facecolor='none',
                     transform=ax1.get_transform('fk5'))
    ax1.add_patch(r)

0.2071786547561607
0.2071786547561607


In [15]:
AG = np.zeros(wcs.pixel_shape, dtype=float)

imax, jmax = wcs.pixel_shape

for i in tqdm(range(imax)):
    for j in range(jmax):
        reg_cen = wcs.pixel_to_world(j, i)
        if reg_cen.separation(center) > 50*u.arcmin:
            AG[i,j] = np.nan
            continue
        reg_data, reg_coords = stars_from_region(data, coords, reg_cen, reg_size)
        logN, m, tot = hist_from_data(reg_data, 20)
        if tot > 0:
            AG[i, j] = np.log10(n/tot)/b
        else:
            AG[i,j] = np.nan

100%|█████████████████████████████████████████| 120/120 [00:48<00:00,  2.49it/s]


In [16]:
# need to inverse array so that it cossesponds to the aladin axis orientation
fig1, ax1 = plt.subplots(subplot_kw=dict(projection=wcs))
# ax.imshow(density[:,::-1], origin='lower')
ax1.imshow(AG, origin='lower')